In [1]:
import sys
print(sys.version)

3.7.9 (default, Aug 31 2020, 07:22:35) 
[Clang 10.0.0 ]


### import

In [1]:
# 標準ライブラリ
import os
import datetime as dt
from datetime import timedelta
from statistics import mean, median,variance,stdev
import itertools
# サードパーティライブラリ
import numpy as np
import pandas as pd
import burst_detection as bd

### 連続した日付データ作成 date_range()
閉区間[start, end]の範囲で出力する

In [2]:
def date_range(start, end):
    for n in range((end - start).days + 1):
        yield start + timedelta(n)

## 変数 定数

In [3]:
# gamma: 
# 基本的に値が高ければ高いほど、バーストが立ち上がりにくくなるパラメータなのでこれの調整の優先度は低いと思います。
# とりあえずgamma＝1とかに設定しておくのがいいです。
# s: 
# 値が高ければ高いほど、バーストが分割される傾向にあるので、1.1~2の範囲で試してみるのがいいかも
param_s = 1.5# 2.0
param_gamma = 1.0 # 1.0

# RESULT_DIR = "/Users/daigo/workspace/koyo/result/"
RESULT_DIR = "/Users/user/Desktop/koyo/result_01/"

total_count_dir = RESULT_DIR + "00total_count/"
rtweets_count_dir = RESULT_DIR + "02rtweets_count/"

out_path = f"{RESULT_DIR}graph/s{param_s}gamma{param_gamma}/"

# out_path = f"{RESULT_DIR}graph/sakura/s{param_s}gamma{param_gamma}/"
# sakura を推定するときはout_path注意。

RATES = range(10, 101, 10)
prefs = ['tk', 'hk', 'is'] # ['tk', 'hk', 'is']
flags = ["icho", "kaede", "sonota", "koyo"]# ["insec"] ["koyo_icho"]["ukoyo1_icho"]  # ["icho", "kaede", "sonota", "koyo", "jumoku"] , "sakura"

In [4]:
CORRECTS = {
    "tk": {
        "kaede": {"start": dt.date(2015, 12, 4), "end": dt.date(2015, 12, 12)},
        "icho": {"start": dt.date(2015, 11, 30), "end": dt.date(2015, 12, 11)},
        "sakura": {"start": dt.date(2015, 3, 23), "end": dt.date(2015, 3, 29)},
    },
    "hk": {
        "kaede": {"start": dt.date(2015, 10, 29), "end": dt.date(2015, 11, 29)},
        "icho": {"start": dt.date(2015, 11, 2), "end": dt.date(2015, 11, 12)},
        "sakura": {"start": dt.date(2015, 4, 22), "end": dt.date(2015, 4, 26)}
    },
    "is": {
        "kaede": {"start": dt.date(2015, 11, 22), "end": dt.date(2015, 12, 3)},
        "icho": {"start": dt.date(2015, 11, 4), "end": dt.date(2015, 11, 10)},
        "sakura": {"start": dt.date(2015, 3, 31), "end": dt.date(2015, 4, 4)}
    }
}

end = dt.date(2015, 12, 31)

sakura_dates = [date for date in date_range(dt.date(2015, 2, 17), end) ]
koyo_short_dates = [date for date in date_range(dt.date(2015, 10, 1), end)]                  
koyo_long_dates = [date for date in date_range(dt.date(2015, 8, 15), end)]

# dates = koyo_long_dates  # ここを変えれば期間が変わる（ようにする予定）

## 関数宣言

### バースト検出

In [5]:
def burst_detection(target_counts, total_counts):
    #number of target events at each time point
    r = np.array(target_counts, dtype=float)
    
    #total number of events at each time point
    d = np.array(total_counts, dtype=float)
    
    #number of time points
    n = len(r)

    q, d, r, p = bd.burst_detection(r,d,n,s=param_s, gamma=param_gamma, smooth_win=1)
    
    """
    bursts = bd.enumerate_bursts(q, 'burstLabel')
    
    #find weight of bursts
    weighted_bursts = bd.burst_weights(bursts,r,d,p)
    
    print('observed probabilities: ') 
    print(str(r/d) )
    print( 'optimal state sequence: ')
    print(str(q.T) )
    print( 'baseline probability: ' + str(p[0]) )
    print( 'bursty probability: ' + str(p[1]) )
    """
    
    return(q)

### 見頃plot

In [6]:
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
from matplotlib.dates import date2num

def plot_migoro(dates, q, pref, flag, rate):
    
    if flag == "sakura":
        x = dates[0:90]
        q = q[0:90]
    else: 
        x = dates
    
    y_correct = []
    if flag == "sakura":
        for date in x:
            if CORRECTS[pref][flag]['start'] <= date <= CORRECTS[pref][flag]['end']:
                y_correct.append(1)
            else:
                y_correct.append(0)
    elif flag == "sonota" or flag == "koyo" or flag == "jumoku" or flag == "koyo_icho":
        for date in x:
            if ( CORRECTS[pref]["kaede"]['start'] <= date <= CORRECTS[pref]["kaede"]['end']
                or CORRECTS[pref]["icho"]['start'] <= date <= CORRECTS[pref]["icho"]['end']):
                y_correct.append(1)
            else:
                y_correct.append(0)
    else: 
        for date in x:
            if  CORRECTS[pref][flag]['start'] <= date <= CORRECTS[pref][flag]['end']:
                y_correct.append(1)
            else: 
                 y_correct.append(0)
                    
    figure_ = plt.figure()  # Figure作成
    figure_.text(0.2, 0.5, str(pref) + ": " + str(flag), fontsize=20)
    figure_.text(0.2, 0.4, "rate = " + str(rate), fontsize=20)
    
    axes_ = figure_.add_subplot(111)  # Axes作成
    xaxis_ = axes_.xaxis  # XAxis取得
    
    x_numdate = date2num([
        dt.date(2015, 8, 15), 
        dt.date(2015, 9, 1), dt.date(2015, 9, 15), 
        dt.date(2015, 10, 1), dt.date(2015, 10, 15), 
        dt.date(2015, 11, 1), dt.date(2015, 11, 15), 
        dt.date(2015, 12, 1), dt.date(2015, 12, 15), dt.date(2015, 12, 31)
    ])
    if flag == "sakura":
        x_numdate = date2num([
        dt.date(2015, 2, 17), 
        dt.date(2015, 3, 15), dt.date(2015, 4, 1), 
        dt.date(2015, 4, 15), dt.date(2015, 5, 1), 
        dt.date(2015, 5, 15)
    ])
    
    axes_.xaxis.set_major_locator(ticker.FixedLocator(x_numdate))
    axes_.tick_params(axis='x', rotation=270)
    xaxis_.set_major_formatter(DateFormatter('%m-%d'))
    
    axes_.plot(x, y_correct, label='correct')
    axes_.plot(x, q, label='burst')
    plt.legend()
    

    fname = f"{flag}_{str(rate).zfill(3)}.png"
    
    os.makedirs(out_path + f"{pref}/", exist_ok=True)
    plt.savefig(out_path + f"{pref}/" + fname, format="png", dpi=300)
    # plt.show()
    
    plt.close()

### F値算出<br>evaluate(result, pref, flag)

In [7]:
def evaluate(result, pref, flag):        
    """
    tp: true positives
    tn: true negatives
    fn: false negativess
    fp: false positives
    """
    tp, tn, fn, fp = 0, 0, 0, 0
    if flag in ["icho", "kaede", "ukoyo1_icho", "insec"]:
        correct_period = [date for date in date_range(CORRECTS[pref][flag]['start'], \
                                                      CORRECTS[pref][flag]['end'])]
    elif flag in ["jumoku", "sonota", "koyo", "koyo_icho"]: 
        icho_period = [date for date in date_range(CORRECTS[pref]['icho']['start'], \
                                                      CORRECTS[pref]['icho']['end'])]
        kaede_period = [date for date in date_range(CORRECTS[pref]['kaede']['start'], \
                                                      CORRECTS[pref]['kaede']['end'])]
        
        correct_period = icho_period + kaede_period
        correct_period = list(set(correct_period))
    elif flag in ["sakura"]:
        correct_period = [date for date in date_range(CORRECTS[pref][flag]['start'], \
                                                      CORRECTS[pref][flag]['end'])]
        

    for oneday in dates:
        if oneday in correct_period:
            if result[result['date']==oneday].iloc[0, 1][0] == 1:
                tp += 1
            elif result[result['date']==oneday].iloc[0, 1][0] == 0:
                tn += 1
            else:
                print("error")
        else:
            if result[result['date']==oneday].iloc[0, 1][0] == 1:
                fp += 1
            elif result[result['date']==oneday].iloc[0, 1][0] == 0:
                fn += 1
            else:
                print("error")
    
    if tp == 0:
        recall = precision = fscore = 0
    else:
        recall = tp / len(correct_period)
        precision = tp / (tp + fp)
        fscore = (2 * recall * precision) / (recall + precision)
    
    # 評価値を出す
#     print(f"recall: {recall}")
#     print(f"precision: {precision}")
    print(f"F-score: {fscore}")
    
    return recall, precision, fscore

### F値plot<br>plot_fscores(fscores, rates, pref, flag):

In [8]:
def plot_fscores(fscores, rates, pref, flag):
    figure_ = plt.figure()  # Figure作成
    axes_ = figure_.add_subplot(111)  # Axes作成
    axes_.plot(rates, fscores, label='fscores')
    
#     out_dir = f"{RESULT_DIR}graph/s{param_s}gamma{param_gamma}/{pref}/"
    fname = f"fscores_{flag}.png"
    
    plt.savefig(out_path + f"{pref}/" + fname, format="png", dpi=300)
    plt.show()
    
    plt.close()

### F値すべてplot<br>plot_all_fscores(results, pref):

In [9]:
def plot_all_fscores(results, pref):
    figure_ = plt.figure()  # Figure作成
    axes_ = figure_.add_subplot(111)  # Axes作成
    
#     flags = ["icho", "kaede", "sonota", "koyo", "jumoku"]
    for flag in flags:
        axes_.plot(RATES, results.query(f"flag == '{flag}'")["fscore"], label=flag)
    
#     out_dir = f"{RESULT_DIR}graph/s{param_s}gamma{param_gamma}/{pref}/"
    fname = f"all_fscores_{pref}.png"
    
    plt.legend()
    plt.savefig(out_path + f"{pref}/" + fname, format="png", dpi=300)
    plt.show()
    
    plt.close()

結果1日単位で書き込む<br>save_burst_date(dates, q, pref, flag, rate)

In [10]:
def save_burst_date(dates, q, pref, flag, rate):
    dir_path = RESULT_DIR + "busrt_dates/"
    f_name = f"{pref}_{flag}_{rate}.csv"
    
    os.makedirs(dir_path, exist_ok=True)
    
    with open(dir_path + f_name, mode='w') as f:
        for i, date in enumerate(dates):
            f.write(f"{date}, {q[i]}\n")

# main関数

### 格納する変数用意

In [11]:
target_counts = dict()
total_counts = dict()

for pref in prefs:
    target_counts[pref] = {}
    for flag in flags:
        target_counts[pref].update({flag: [ [], [], [], [], [], [], [], [], [], [] ]})
        
target_counts[prefs[0]]

{'icho': [[], [], [], [], [], [], [], [], [], []],
 'kaede': [[], [], [], [], [], [], [], [], [], []],
 'sonota': [[], [], [], [], [], [], [], [], [], []],
 'koyo': [[], [], [], [], [], [], [], [], [], []]}

### ファイル読み込み

In [12]:
for pref, flag in  itertools.product(prefs, flags):
    for i, rate in enumerate(RATES):
        filename = f"{pref}_{flag}_{str(rate).zfill(3)}rwords_count.txt"
        with open(rtweets_count_dir+filename, "r") as target_file:
             target_counts[pref][flag][i] = [line.rstrip('\n').split('\t')[1] for line in target_file.readlines()]

total_counts = {}   
for pref in prefs:
    fname = pref + "_total_dailycount.txt"
    total_counts[pref] = pd.read_csv(total_count_dir + fname, names=("date", "count"), header=None, sep='\t')

### 処理部

In [13]:
for pref, flag in itertools.product(prefs, flags):
    results = pd.DataFrame(columns=["rate", "fscore", "recall", "precision"])
    print(f"{pref} -------------{flag}")
    for i, rate in enumerate(RATES):
        
        if flag == "sakura":
            q = burst_detection(target_counts[pref][flag][i], total_counts[pref]["count"] )
        else:
            total = list(total_counts[pref].query("date >= '2015-08-15'")["count"].values)
            target = target_counts[pref][flag][i]
            if np.all(np.array(target) == '0'): # targetが全て0だとburst_detectionがエラーになる。
                q = []
                for i in range(len(target)):
                    q.append([0.0])
            else:
                q = burst_detection(target, total)
                
        if flag == "sakura":
            dates = sakura_dates
        else:
            dates = koyo_long_dates
        
        save_burst_date(dates, q, pref, flag, rate)
        plot_migoro(dates, q, pref, flag, rate)

        q_dateframe = pd.DataFrame([dates, q]).T
        q_dateframe.columns = ['date', 'burst']
        
        recall, precision, fscore = evaluate(q_dateframe, pref, flag)
        
        result = pd.Series([rate, fscore, recall, precision], index=["rate", "fscore", "recall", "precision"])
        results = results.append(result, ignore_index=True)
        
    target = flag
    os.makedirs(f"./result_01/scores/{target}/", exist_ok=True)
    results.to_csv(f"./result_01/scores/{target}/{pref}_{flag}.csv")  
    
print("---------\n終了\n---------\n")

tk -------------icho
F-score: 0.5581395348837209
F-score: 0.5581395348837209
F-score: 0.5454545454545454
F-score: 0.5581395348837209
F-score: 0.5581395348837209
F-score: 0.5217391304347826
F-score: 0.5333333333333333
F-score: 0.5333333333333333
F-score: 0.5217391304347826
F-score: 0.5333333333333333
tk -------------kaede
F-score: 0
F-score: 0
F-score: 0
F-score: 0
F-score: 0
F-score: 0
F-score: 0
F-score: 0
F-score: 0
F-score: 0
tk -------------sonota
F-score: 0.896551724137931
F-score: 0.7027027027027027
F-score: 0.6666666666666666
F-score: 0.7027027027027027
F-score: 0.6842105263157895
F-score: 0.65
F-score: 0.65
F-score: 0.5531914893617021
F-score: 0.5531914893617021
F-score: 0.5531914893617021
tk -------------koyo
F-score: 0.5777777777777777
F-score: 0.5909090909090909
F-score: 0.5531914893617021
F-score: 0.5531914893617021
F-score: 0.5909090909090909
F-score: 0.5909090909090909
F-score: 0.5652173913043478
F-score: 0.5531914893617021
F-score: 0.5531914893617021
F-score: 0.541666666